In [1]:
import torch
import torch.nn as nn

In [ ]:
class MultiheadCausalAttention(nn.Module):
    super().__init__()
    def __init__(self, d_in, d_out, context_length, dropout,n_heads,qkv_bias=False):
        assert (d_out % n_heads == 0), \
    "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = n_heads
        # self.head_dim = d_out // num_heads 
        self.head_dim = d_out // n_heads
        
        self.query_w = torch.nn.linear(d_in, d_out, bias=qkv_bias)
        self.key_w = torch.nn.linear(d_in, d_out, bias=qkv_bias)
        self.value_w = torch.nn.linear(d_in, d_out, bias=qkv_bias)
        
        self.register_buffer('mask', torch.trilu(torch.ones(context_length, context_length), diagonal=1))